In [1]:
# Install PyTorch to build the network below
%pip install torch

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Install shark-turbine to convert torch to linalg
%pip install shark-turbine

Note: you may need to restart the kernel to use updated packages.


In [5]:
import torch
import torch.nn as nn


class MyAdd(nn.Module):
    def __init__(self):
        super(MyAdd, self).__init__()

        self.add_values = nn.Parameter(torch.DoubleTensor([1.0, 2.0, 3.0]))

    def forward(self, x):
        # Ensure the input tensor is of double precision
        x = x.double()

        # Add the constant values to the input tensor
        result = x + self.add_values

        return result

# Example usage
input_tensor = torch.randn(3, dtype=torch.float64)  # Replace with your input tensor
my_add_module = MyAdd()

output_tensor = my_add_module(input_tensor)
print("Input Tensor:", input_tensor)
print("Output Tensor:", output_tensor)

Input Tensor: tensor([ 0.2244, -0.2551, -0.3616], dtype=torch.float64)
Output Tensor: tensor([1.2244, 1.7449, 2.6384], dtype=torch.float64, grad_fn=<AddBackward0>)


In [19]:
import shark_turbine.aot as aot
from iree.compiler import compile_str

example_x = torch.empty((3,), dtype=torch.float64)
exported = aot.export(my_add_module, example_x)
with open("add_torch.mlir", "w") as f:
  exported.mlir_module.print(file=f)

In [20]:
!iree-compile --iree-input-type=torch --compile-to=input add_torch.mlir > add_linalg.mlir